In [1]:
# pip install transformers datasets rouge-score nltk bert-score
import pandas as pd
import csv
import random
from collections import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [48]:
pred_df = pd.read_csv('/kaggle/input/generated-endings/test_predictions.csv')

In [49]:
pred_df.head()

,Input,Target,Generated
0,Complete this story by generating its last lin...,The family left the restaurant without eating.,Complete this story by generating its last lin...
1,Complete this story by generating its last lin...,His daughter loved them and used them to teach...,Complete this story by generating its last lin...
2,Complete this story by generating its last lin...,Joe gave in and now they have a new member of ...,Complete this story by generating its last lin...
3,Complete this story by generating its last lin...,Now my neck is sore but I feel better.,Complete this story by generating its last lin...
4,Complete this story by generating its last lin...,There were many wrecks.,Complete this story by generating its last lin...


In [50]:
pred_df['Generated']

0       Complete this story by generating its last lin...
1       Complete this story by generating its last lin...
2       Complete this story by generating its last lin...
3       Complete this story by generating its last lin...
4       Complete this story by generating its last lin...
                              ...                        
1995    Complete this story by generating its last lin...
1996    Complete this story by generating its last lin...
1997    Complete this story by generating its last lin...
1998    Complete this story by generating its last lin...
1999    Complete this story by generating its last lin...
Name: Generated, Length: 2000, dtype: object

In [51]:
# Splitting the 'story' column into lists of sentences
pred_df['sentences'] = pred_df['Generated'].str.split('\.')

# Extracting the fifth sentence
pred_df['ending'] = pred_df['sentences'].apply(lambda x: x[4].strip() if len(x) > 4 else "I don't know.")

# Dropping the intermediate 'sentences' column as it's no longer needed if you wish
pred_df.drop(columns=['sentences'], inplace=True)


In [52]:
pred_df

,Input,Target,Generated,ending
0,Complete this story by generating its last lin...,The family left the restaurant without eating.,Complete this story by generating its last lin...,The dad yelled at the kids
1,Complete this story by generating its last lin...,His daughter loved them and used them to teach...,Complete this story by generating its last lin...,He bought them for her
2,Complete this story by generating its last lin...,Joe gave in and now they have a new member of ...,Complete this story by generating its last lin...,Joe agreed
3,Complete this story by generating its last lin...,Now my neck is sore but I feel better.,Complete this story by generating its last lin...,I woke up again and decided to go to the bathroom
4,Complete this story by generating its last lin...,There were many wrecks.,Complete this story by generating its last lin...,They were so tired
...,...,...,...,...
1995,Complete this story by generating its last lin...,The drug connect now has a hit out on Ghost.,Complete this story by generating its last lin...,Ghost is going to kill the drug connect
1996,Complete this story by generating its last lin...,Nick is much happier with his new device.,Complete this story by generating its last lin...,Nick was very happy with the new phone
1997,Complete this story by generating its last lin...,I took the boat out on the lake and felt happy.,Complete this story by generating its last lin...,I bought it
1998,Complete this story by generating its last lin...,She was overjoyed with the results of her choi...,Complete this story by generating its last lin...,She decided to try out for the team


In [53]:
for index, row in pred_df.head(5).iterrows():
    print(f"{row['Input']}\n")
    print(f"{row['ending']}\n")
    print(f"Target: {row['Target']}")
    print("--------------------------------------------------- \n")  # Adding a separator for clarity

Complete this story by generating its last line to give it a logical ending:
 The family went out to eat. The kids were being rowdy. The dad told the kids to calm down. The kids continued to be rowdy.

The dad yelled at the kids

Target: The family left the restaurant without eating.
--------------------------------------------------- 

Complete this story by generating its last line to give it a logical ending:
 George's daughter had recently taken a huge interest in ballet. He didn't have the money to afford nice lessons. Still, for her birthday he wanted to do something nice. He found her a great pair of ballet slippers at a specialty shop.

He bought them for her

Target: His daughter loved them and used them to teach herself to dance.
--------------------------------------------------- 

Complete this story by generating its last line to give it a logical ending:
 Joe took his family to the pet store to pick up dog food. His kids explored the store and found the adoption area. The

In [54]:
references=[]
predictions = []
for index, instance in pred_df.iterrows():
    references.append(instance['Target'])
    predictions.append(instance['ending'])

In [55]:
len(references), len(predictions)

(2000, 2000)

In [1]:
# !pip install -q evaluate bert_score rouge meteor bleu perplexity
# !pip install bert_score

In [2]:
# pip install evaluate

In [12]:
from evaluate import load

2024-04-25 21:20:00.229163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 21:20:00.229317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 21:20:00.357688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [56]:
# Calculates the BERT score for the random selection model
bertscore = load("bertscore")
results_bert = bertscore.compute(predictions=predictions, references=references, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
import numpy as np

In [59]:
np.mean(results_bert['f1'])

0.8778570618629455

In [3]:
# pip install --upgrade nltk

In [20]:
# from nltk.translate.meteor_score import meteor_score
import nltk
# nltk.download('all')
nltk.download('wordnet')
# Download specific resource
nltk.download('wordnet')

# Optionally, download the Open Multilingual WordNet, if you're using newer versions of NLTK
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [21]:
# Calculates the METEOR score for the random selection model
meteor = load('meteor')
results_meteor = meteor.compute(predictions=predictions, references=references)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [15]:
results_meteor

{'meteor': 0.10026983864382363}

In [23]:
# Calculates the BLEU score
bleu = load("bleu")
result_bleu = bleu.compute(predictions=predictions, references=references)

In [18]:
result_bleu

{'bleu': 0.012528953869536804,
 'precisions': [0.19210451170759565,
  0.034804329725228976,
  0.008388256440982624,
  0.003236245954692557],
 'brevity_penalty': 0.6070071344074278,
 'length_ratio': 0.667015856387791,
 'translation_length': 14008,
 'reference_length': 21001}

In [4]:
# pip install rouge_score

In [26]:
rouge = load('rouge')
results_rouge = rouge.compute(predictions=predictions, references=references)

In [22]:
results_rouge

{'rouge1': 0.1801753968747533,
 'rouge2': 0.03219242937622962,
 'rougeL': 0.16953743632130916,
 'rougeLsum': 0.1696750144207242}

In [36]:
filtered_pred, filtered_ref = zip(*[(p, r) for p, r in zip(predictions, references) if p != ""])

# Convert the results back to lists if necessary
filtered_pred = list(filtered_pred)
filtered_ref = list(filtered_ref)

In [35]:
# # # Calculates the Perplexity score for the predictions from the random selection model
# perplexity = load("perplexity", module_type="metric")
# results_perplexity = perplexity.compute(predictions=predictions, model_id='gpt2')

In [46]:
perplexity = load("perplexity", module_type="metric")
results_perplexity = perplexity.compute(predictions=filtered_pred, model_id='state-spaces/mamba-370m-hf') #for test_pred_2

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/124 [00:00<?, ?it/s]

In [39]:
np.mean(results_perplexity['perplexities'])

171.74454909973298